<a href="https://colab.research.google.com/github/LorenzEh/LLM-Vaccine-Hesitency/blob/main/LLM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the libraries


In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes trl accelerate llama-cpp-python huggingface-hub

# Getting the data ready



1.   Download the imputed data from google drive
2.   Exclude most of the columns (since we're running the code on CPU keeping all of the columns is not feasible)
3. Convert pandas dataframe to dictionary, so that it can be converted to a Hugging Face dataset in the next part



In [ ]:
import gdown
import pandas as pd
from datasets import Dataset

In [ ]:
gdown.download(id="1VXUSmDyt7qYo5nGpMDWmelTjnWk6KVkb", output="data_imputed_decoded.csv", quiet=False) # unfortunatly the csv does not work, therefore we need to continue with the stata format
data = pd.read_csv("data_imputed_decoded.csv") # copy and paste a single


In [ ]:
for i in data.columns: # print all column names
    print(i)

In [ ]:
# list of prefixes to include
prefixes_to_include = ["AGE_CAT", "EDU_CAT", "BULA", "GENDER", "Aussage", "Impfen"]

# function to filter columns based on prefixes
def filter_columns_by_prefix(df, prefixes):
    filtered_columns = [col for col in df.columns if any(col.startswith(prefix) for prefix in prefixes)]
    return df[filtered_columns]

# create the new DataFrame with the filtered columns
new_df = filter_columns_by_prefix(data, prefixes_to_include)

In [ ]:
for i in new_df.columns:
    print(i)

In [ ]:
new_df.head(10)

In [ ]:
new_df['EDU_CAT'] = new_df['EDU_CAT'].str.replace(r' \(Kat\. \d+(-\d+)?\)', '', regex=True) # delete the "(Kat )" in EDU_CAT

In [ ]:
# Antworten in Text-Format bringen
def create_prompt(row):
    prompt = ""
    for question, answer in row.items():
        prompt += f'Aussage: "{question}" → Antwort: "{answer}"\n'
    return {"text": prompt.strip()}

# Anwendung auf jede Zeile
prompts = new_df.apply(create_prompt, axis=1)

In [ ]:
prompts.head(10)

# Convert dictionary data to sentences using local LLM


1.   Parse the dictionary, to make it ready for the chat template
2.   Introduce a chat template, this helps the model greatly to understand the input
3.   



In [ ]:
import re
import random
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# set random seed for reproducibility
random.seed(42)
torch.manual_seed(42)

In [ ]:
# parsing of the raw-data
def parse_entry(entry):
    text = entry["text"]

    # split into key-value pairs
    pairs = re.split(r'\nAussage: ', text)
    persona_data = {}
    for pair in pairs:
        if not pair.strip():
            continue
        match = re.match(r'^"(.*?)" → Antwort: "(.*?)"', pair)
        if match:
            key = match.group(1).replace("Aussage: ", "").strip()
            value = match.group(2).strip()
            persona_data[key] = value

    # build prompt using key demographics
    prompt = f"""Person:
- Geschlecht: {persona_data.get('GENDER', 'Unbekannt')}
- Alter: {persona_data.get('AGE_CAT', 'Unbekannt')}
- Bildung: {persona_data.get('EDU_CAT', 'Unbekannt')}
- Bundesland: {persona_data.get('BULA', 'Unbekannt')}
- Impfen Ehestmöglich: {persona_data.get('Impfen: Ehestmöglich', 'Unbekannt')}
"""
    response = "[PLACEHOLDER_FOR_GENERATED_PERSONA]"
    return {"prompt": prompt, "response": response}

In [ ]:
# we're getting the data ready by creating a prompt for each person, which can easly be interpreted by the LLM
data_dict = {"prompt": [], "response": []}
for entry in prompts:
    parsed = parse_entry(entry)
    data_dict["prompt"].append(parsed["prompt"])
    data_dict["response"].append(parsed["response"])

dataset = Dataset.from_dict(data_dict)
dataset = dataset.select(range(5)) # we're limiting the dataset size , since we're working on a cpu

print(dataset["prompt"][0]) # description of person which is used in the prompt
print(dataset["response"][0]) # placeholder

In [ ]:
# Schritt 2: Modell von Hugging Face herunterladen
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(
    repo_id="TheBloke/em_german_mistral_v01-GGUF",
    filename="em_german_mistral_v01.Q4_K_M.gguf",
    local_dir="/content"  # Speicherort im Colab-Filesystem
)

# Schritt 3: Modell initialisieren
from llama_cpp import Llama

llm = Llama(
    model_path=model_path,  # Verwendet den automatischen Pfad
    n_ctx=2048,
    n_threads=8,           # Nutzt alle CPU-Kerne
    n_gpu_layers=0         # 0 = Nur CPU (für GPU: 35+ setzen)
)


In [ ]:
# test the model (...)
# output = llm("Was ist eine Impfung?", max_tokens=100)
# print(output["choices"][0]["text"])

In [ ]:
FEW_SHOT_EXAMPLES = [
    {
        "input": """Person:
- Geschlecht: Weiblich
- Alter: 30-39 Jahre
- Bildung: hoch
- Bundesland: Wien
- Impfen Ehestmöglich: Trifft eher zu""",
        "output": "Die 30-39-jährige hat einen hohen Bildungsstand, kommt aus Wien und würde sich eher zeitnah impfen lassen."
    },
    {
        "input": """Person:
- Geschlecht: Männlich
- Alter: 60-69 Jahre
- Bildung: mittel
- Bundesland: Tirol
- Impfen Ehestmöglich: Trifft eher nicht zu""",
        "output": "Der 60-69-jährige Mann aus Tirol hat einen mittleren Bildungsstand und wird sich eher nicht zeitnah impfen lassen."
    },
    {
        "input": """Person:
- Geschlecht: Weiblich
- Alter: 18-29 Jahre
- Bildung: niedrig
- Bundesland: Steiermark
- Impfen Ehestmöglich: Trifft überhaupt nicht zu""",
        "output": "Die 18-29-jährige Steirerin hat einen niedrigen Bildungsstand und lehnt eine COVID-19-Impfung grundsätzlich ab."
    },
    {
        "input": """Person:
- Geschlecht: Weiblich
- Alter: 18-29 Jahre
- Bildung: niedrig
- Bundesland: Oberösterreich
- Impfen Ehestmöglich: Teils-teils""",
        "output": "Die 18-29-jährige Oberösterreicherin hat einen niedrigen Bildungsstand und weiß nicht, ob sie sich ehestmöglich impfen lassen würde."
    }
]

def format_prompt(prompt):
    instruction = """[INST] <<SYS>>
Du bist ein deutscher Demographie-Experte. Fasse Personendaten in einem Satz zusammen:
- Format: [Alter]-jährige/r [Bildung] [Geschlecht] aus [Bundesland] [Impfverhalten]
- Beispiel: "Die 30-39-jährige Akademikerin aus Wien wird sich ehestmöglich impfen lassen."
- Nur die gegebenen Daten verwenden!
<</SYS>>"""

    examples = "\n\n".join(
        f"{ex['input']}\nZusammenfassung: {ex['output']}"
        for ex in FEW_SHOT_EXAMPLES
    )

    return f"""
{instruction}

Beispiele:
{examples}

Aktuelle Daten:
{prompt}
Zusammenfassung: [/INST]"""  # Moved [/INST] to end

In [ ]:
def generate_persona(llm, prompt):
    full_prompt = format_prompt(prompt)

    output = llm(
        full_prompt,
        max_tokens=100,        # Increased from 50
        temperature=0.6,       # More creativity
        top_p=0.95,            # Wider sampling
        top_k=40,              # More diverse
        repeat_penalty=1.1,    # Reduced penalty
        stop=["\n\n"],         # Simpler stop condition
        echo=False             # Don't return prompt
    )

    # Handle empty response
    response = output['choices'][0]['text'].strip()

    # Fallback if empty
    if not response:
        return "Zusammenfassung konnte nicht generiert werden."

    return response.split("Zusammenfassung:")[-1].strip()


In [ ]:
# Testlauf für die ersten 5 Einträge
for i in range(5):
    prompt = dataset[i]["prompt"]
    response = generate_persona(llm, prompt)

    print(f"Prompt {i+1}:")
    print(prompt)
    print("\nGenerierte Persona:")
    print(response)
    print("\n" + "="*50 + "\n")

# Rest

In [ ]:
import re
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# --- Konfiguration ---
MODEL_NAME = "dbmdz/german-gpt2"
DEVICE = "cpu"

GENERATION_CONFIG = {
    "max_new_tokens": 50,
    "do_sample": True,
    "temperature": 0.7,
    "top_k": 40,
    "top_p": 0.9,
    "repetition_penalty": 1.5,
    "no_repeat_ngram_size": 2,
    "stop_sequence": "\n"
}

FEW_SHOT_EXAMPLES = [
    {
        "input": """Person:
- Geschlecht: Weiblich
- Alter: 30-39 Jahre
- Bildung: Hoch
- Bundesland: Wien
- Impfen Ehestmöglich: Trifft eher zu""",
        "output": "Die 30-39-jährige hochgebildete Frau aus Wien tendiert zu einer baldigen Impfung."
    },
    {
        "input": """Person:
- Geschlecht: Männlich
- Alter: 60-69 Jahre
- Bildung: Mittel
- Bundesland: Tirol
- Impfen Ehestmöglich: Trifft eher nicht zu""",
        "output": "Der 60-69-jährige Mann mit mittlerer Bildung aus Tirol steht Impfungen skeptisch gegenüber."
    }
]

# --- Datenverarbeitung ---
def parse_entry(entry: dict) -> dict:
    """Verarbeitet Rohdaten zu einem strukturierten Prompt"""
    text = entry["text"]

    # Extrahiere Key-Value Paare
    persona_data = {}
    for pair in re.split(r'\nAussage: ', text):
        if match := re.match(r'^"(.*?)" → Antwort: "(.*?)"', pair):
            key = match.group(1).replace("Aussage: ", "").strip()
            persona_data[key] = match.group(2).strip()

    # Validiere erforderliche Felder
    required_fields = ['GENDER', 'AGE_CAT', 'EDU_CAT', 'BULA', 'Impfen: Ehestmöglich']
    for field in required_fields:
        if field not in persona_data:
            persona_data[field] = 'Unbekannt'

    # Erstelle Prompt
    prompt = f"""Person:
- Geschlecht: {persona_data['GENDER']}
- Alter: {persona_data['AGE_CAT']}
- Bildung: {persona_data['EDU_CAT']}
- Bundesland: {persona_data['BULA']}
- Impfen Ehestmöglich: {persona_data['Impfen: Ehestmöglich']}"""

    return {"prompt": prompt, "response": ""}

def create_prompt(example: dict) -> str:
    """Erstellt den finalen Prompt mit Few-Shot Beispielen"""
    system_prompt = """Erstelle eine Personenbeschreibung in einem Satz nach diesen Regeln:
1. Beginne mit dem Alter (z.B. "30-39-jährige")
2. Verwende nur die angegebenen Informationen
3. Struktur: [Alter] [Bildung] [Geschlecht] aus [Bundesland] [Impfverhalten]"""

    example_template = """\
Beispiel-Eingabe:
{input}

Beispiel-Ausgabe:
{output}"""

    examples = "\n\n".join(
        example_template.format(**ex)
        for ex in FEW_SHOT_EXAMPLES
    )

    return f"""\
{system_prompt}

{examples}

Eingabe:
{example['prompt']}

Ausgabe:"""

# --- Modelloperationen ---
def initialize_model() -> pipeline:
    """Initialisiert das Modell und den Tokenizer"""
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float32
    ).to(DEVICE)

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token

    return pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device=DEVICE
    )

def clean_response(text: str) -> str:
    """Bereinigt die generierte Antwort"""
    # Entferne unerwünschte Muster
    text = re.sub(r'<\|.*?\|>|<\/s>|\[.*?\]', '', text)
    text = re.sub(r'\b\d+\+?\b', '', text)

    # Erzwinge korrekte Formatierung
    if match := re.search(r'(\d+-\d+)\s*Jahre', text):
        text = text.replace(match.group(0), f"{match.group(1)}-jährige")

    return text.strip(". \n")

# --- Hauptprozess ---
def main():
    # 1. Daten vorbereiten
    raw_data = [{"prompt": entry} for entry in dataset]  # Ihre Rohdaten
    dataset = Dataset.from_dict({"prompt": raw_data})
    dataset = dataset.map(parse_entry)

    # 2. Modell initialisieren
    pipe = initialize_model()

    # 3. Generierung durchführen
    def generate(example):
        prompt = create_prompt(example)
        output = pipe(prompt, **GENERATION_CONFIG)[0]['generated_text']
        response = output.split("Ausgabe:")[-1].split("\n")[0].strip()
        example["response"] = clean_response(response)
        return example

    dataset = dataset.map(generate)

    # 4. Ergebnisse anzeigen
    for idx in range(3):
        print("\nPrompt:")
        print(dataset[idx]["prompt"])
        print("\nGenerierte Antwort:")
        print(dataset[idx]["response"])
        print("="*50)

if __name__ == "__main__":
    main()

In [ ]:
import re
import random
from datasets import Dataset

# Few-shot examples
few_shot_examples = [
    {
        "user": """Analysiere diese Person:
Geschlecht: Weiblich
Alter: 30-39 Jahre
Bildung: Hochschulabschluss
Wohnort: Wien
Impfbereitschaft: Hoch""",
        "assistant": """Die 30-39-jährige Akademikerin aus Wien zeigt eine hohe Bereitschaft zur zeitnahen Impfung."""
    },
    {
        "user": """Analysiere diese Person:
Geschlecht: Männlich
Alter: 60-69 Jahre
Bildung: Berufsausbildung
Wohnort: Tirol
Impfbereitschaft: Gering""",
        "assistant": """Der 60-69-jährige Handwerker aus Tirol steht Impfungen kritisch gegenüber."""
    }
]

# Select relevant few-shot examples
def select_relevant_examples(prompt):
    age_group_match = re.search(r"Alter: (.+?) Jahre", prompt)
    if age_group_match:
        age_group = age_group_match.group(1)
        relevant_examples = [ex for ex in few_shot_examples if age_group in ex["user"]]
        if relevant_examples:
            return [random.choice(relevant_examples)]
    return [random.choice(few_shot_examples)]

# Format into chat messages
def format_chat_template(example, tokenizer=None):
    system_prompt = """Erstelle präzise Personenbeschreibungen in einem Satz nach diesen Regeln:
1. Beginne immer mit dem Alter (z.B. "30-39-jährige")
2. Verwende nur die gegebenen Attribute
3. Struktur: [Alter] [Bildung] [Geschlecht] aus [Ort] [Impfverhalten]
4. Keine Zusatzinformationen erfinden

Beispiel: "Die 30-39-jährige Hochschulabsolventin aus Wien befürwortet eine rasche Impfung."
"""

    selected_examples = select_relevant_examples(example["prompt"])

    messages = [{"role": "system", "content": system_prompt}]
    for ex in selected_examples:
        messages.extend([
            {"role": "user", "content": f"Analysiere:\n{ex['user']}"},
            {"role": "assistant", "content": f"{ex['assistant']}"}
        ])

    messages.append({
        "role": "user",
        "content": f"Analysiere:\n{example['prompt']}\n\nZusammenfassung:"
    })

    # Manual formatting of chat if tokenizer.apply_chat_template fails
    formatted_messages = ""
    for message in messages:
        if message["role"] == "system":
            formatted_messages += f"<|system|>\n{message['content']}\n"
        elif message["role"] == "user":
            formatted_messages += f"<|user|>\n{message['content']}\n"
        elif message["role"] == "assistant":
            formatted_messages += f"<|assistant|>\n{message['content']}\n"
    formatted_messages += "<|assistant|>\n"

    example["text"] = formatted_messages
    return example

# Pretty-print helper
def pretty_print_series_row(dataset, row_number):
    if not isinstance(dataset, Dataset):
        raise ValueError("Input must be a Hugging Face Dataset")
    row = dataset[row_number]
    text = row.get('text', '')
    formatted = text.replace('\\n', '\n').replace('- ', '  - ')
    print(formatted)

# Clean response
def clean_response(response):
    response = re.sub(r'\d+[.,]\d+|\d+-\d+|\d+[/]\d+', '', response)
    response = re.sub(r'\b(Krankenhaus|Medikamente|Survey)\b', '', response, flags=re.I)
    if re.search(r'\d+-\d+', response) and 'jährig' not in response:
        age_match = re.search(r'(\d+-\d+)', response)
        response = f"{age_match.group(1)}-jährige " + response.replace(age_match.group(1), '')
    return response

# Generate persona
def generate_persona(pipe, text, max_new_tokens=60):
    base_info = text.split('Person:')[-1].split('<|assistant|>')[0].strip()
    prompt = f"""Erstelle einen präzisen Satz nach diesem Muster:
Angaben: Geschlecht: Weiblich, Alter: 30-39, Bildung: Hoch, Bundesland: Wien, Impfen: Trifft eher zu
Satz: Die 30-39-jährige hochgebildete Frau aus Wien tendiert zu einer baldigen Impfung.

Verwende diese Angaben:
{base_info}
Satz:"""

    outputs = pipe(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.2,
        top_k=10,
        top_p=0.60,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2,
        num_beams=5
    )

    raw_response = outputs[0]['generated_text'].split("Zusammenfassung:")[-1].strip()
    response = clean_response(raw_response)

    print("Prompt:\n", prompt)
    print("\nAntwort:\n", response)
    return response

# Example Usage:
dataset = dataset.map(lambda example: format_chat_template(example))
# pretty_print_series_row(dataset, 1)



In [ ]:
from transformers import pipeline

# 1. Load a small model pipeline for generation
pipe = pipeline("text-generation", model="gpt2")  # or use your own model

# 2. Example: create your dataset manually
data = {
    "prompt": [
        """Geschlecht: Weiblich
Alter: 30-39 Jahre
Bildung: Hochschulabschluss
Wohnort: Wien
Impfbereitschaft: Hoch"""
    ]
}

from datasets import Dataset
dataset = Dataset.from_dict(data)

# 3. Format your dataset into chat template
dataset = dataset.map(lambda example: format_chat_template(example))

# 4. Pick the row you want
row_number = 0
row = dataset[row_number]

# 5. Get the text
text = row['text']

# 6. Generate the persona sentence
persona = generate_persona(pipe, text)

# 7. Print both
print("\n\nFinal Output:\n")
print("Prompt:")
print(text)
print("\nGenerated Persona:")
print(persona)

In [ ]:


# --- Konfiguration ---
MODEL_NAME = "dbmdz/german-gpt2"
GENERATION_CONFIG = {
    "max_new_tokens": 60,
    "do_sample": True,
    "temperature": 0.4,
    "top_k": 30,
    "top_p": 0.85,
    "repetition_penalty": 1.5,
    "no_repeat_ngram_size": 2,
    "num_beams": 2
}

FEW_SHOT_EXAMPLES = [
    {
        "input": """Person:
- Geschlecht: Weiblich
- Alter: 30-39 Jahre
- Bildung: Hoch
- Bundesland: Wien
- Impfen Ehestmöglich: Trifft eher zu""",
        "output": """Die 30-39-jährige hochgebildete Frau aus Wien tendiert zu einer baldigen Impfung."""
    },
    {
        "input": """Person:
- Geschlecht: Männlich
- Alter: 60-69 Jahre
- Bildung: Mittel
- Bundesland: Tirol
- Impfen Ehestmöglich: Trifft eher nicht zu""",
        "output": """Der 60-69-jährige Mann mit mittlerer Bildung aus Tirol steht einer zeitnahen Impfung skeptisch gegenüber."""
    }
]

# --- Hilfsfunktionen ---
def clean_response(text: str) -> str:
    """Bereinigt die generierte Antwort von unerwünschten Inhalten"""
    # Entferne unerwünschte Muster
    patterns_to_remove = [
        r'<\|.*?\|>',  # Chat-Tags
        r'\d+[.,]\d+|\d+-\d+|\d+[/]\d+',  # Zahlenformate
        r'\b(Krankenhaus|Medikamente|Survey)\b'  # Unerwünschte Begriffe
    ]

    for pattern in patterns_to_remove:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    # Normalisiere Leerzeichen
    text = re.sub(r'\s+', ' ', text).strip()

    # Erzwinge korrekte Altersformatierung
    if age_match := re.search(r'(\d+-\d+)', text):
        if 'jährig' not in text:
            text = f"{age_match.group(1)}-jährige " + text.replace(age_match.group(1), '')

    # Erzwinge Satzende
    if not text.endswith(('.', '!', '?')):
        text = f"{text.rstrip('.')}."

    return text

def select_relevant_example(prompt: str) -> dict:
    """Wählt ein relevantes Few-Shot-Beispiel basierend auf dem Alter aus"""
    if age_match := re.search(r"Alter: (.+?) Jahre", prompt):
        age_group = age_match.group(1)
        return next((ex for ex in FEW_SHOT_EXAMPLES if age_group in ex["input"]), None)
    return random.choice(FEW_SHOT_EXAMPLES)

# --- Hauptfunktionen ---
def format_chat_template(example: dict, tokenizer: AutoTokenizer) -> dict:
    """Formatiert den Beispieltext in ein Chat-Template"""
    system_prompt = """Erstellen Sie eine prägnante Personenbeschreibung in einem Satz:
- Beginne mit dem Alter (z.B. "30-39-jährige")
- Verwende nur die angegebenen Attribute
- Struktur: [Alter] [Bildung] [Geschlecht] aus [Ort] [Impfverhalten]
- Keine zusätzlichen Informationen erfinden"""

    example_data = example["prompt"]
    selected_example = select_relevant_example(example_data)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": selected_example["input"]},
        {"role": "assistant", "content": selected_example["output"]},
        {"role": "user", "content": example_data}
    ]

    example["text"] = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return example

def generate_persona(pipe: pipeline, text: str) -> str:
    """Generiert eine Personenbeschreibung basierend auf den Eingabedaten"""
    base_info = re.search(r"Person:(.*?)<\|assistant\|>", text, re.DOTALL).group(1).strip()

    prompt = f"""Beispiel:
Input: {FEW_SHOT_EXAMPLES[0]['input']}
Output: {FEW_SHOT_EXAMPLES[0]['output']}

Generiere für folgende Daten:
{base_info}
Output:"""

    output = pipe(prompt, **GENERATION_CONFIG)[0]['generated_text']
    response = output.split("Output:")[-1].strip()
    return clean_response(response)

# --- Initialisierung ---
def initialize_pipeline() -> pipeline:
    """Initialisiert die Model-Pipeline"""
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float32
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token

    return pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        model_kwargs={"pad_token_id": tokenizer.eos_token_id}
    )

# --- Hauptablauf ---
if __name__ == "__main__":
    # Annahme: dataset ist bereits definiert
    pipe = initialize_pipeline()
    dataset = dataset.map(lambda ex: format_chat_template(ex, pipe.tokenizer))

    new_texts = []
    for entry in dataset:
        response = generate_persona(pipe, entry["text"])
        updated_text = entry["text"].replace("<|assistant|>", f"<|assistant|>\n{response}</s>")
        new_texts.append(updated_text)

    dataset = dataset.add_column("text_with_response", new_texts)

In [ ]:
few_shot_examples = [
    {
        "user": """Analysiere diese Person:
Geschlecht: Weiblich
Alter: 30-39 Jahre
Bildung: Hochschulabschluss
Wohnort: Wien
Impfbereitschaft: Hoch""",
        "assistant": """Zusammenfassung: Die 30-39-jährige Akademikerin aus Wien zeigt eine hohe Bereitschaft zur zeitnahen Impfung."""
    },
    {
        "user": """Analysiere diese Person:
Geschlecht: Männlich
Alter: 60-69 Jahre
Bildung: Berufsausbildung
Wohnort: Tirol
Impfbereitschaft: Gering""",
        "assistant": """Zusammenfassung: Der 60-69-jährige Handwerker aus Tirol steht Impfungen kritisch gegenüber."""
    }
]



def select_relevant_examples(prompt):
    age_group_match = re.search(r"Alter: (.+?) Jahre", prompt)
    if age_group_match:
        age_group = age_group_match.group(1)
        relevant_examples = [ex for ex in few_shot_examples if age_group in ex["user"]]
        if relevant_examples:
            return [random.choice(relevant_examples)]
    return [random.choice(few_shot_examples)]

def format_chat_template(example, tokenizer):


    system_prompt = """Erstelle präzise Personenbeschreibungen in einem Satz nach diesen Regeln:
1. Beginne immer mit dem Alter (z.B. "30-39-jährige")
2. Verwende nur die gegebenen Attribute
3. Struktur: [Alter] [Bildung] [Geschlecht] aus [Ort] [Impfverhalten]
4. Keine Zusatzinformationen erfinden

Beispielantwort: "Die 30-39-jährige Hochschulabsolventin aus Wien befürwortet eine rasche Impfung."
"""

    selected_examples = select_relevant_examples(example["prompt"])

    messages = [{"role": "system", "content": system_prompt}]

    for ex in selected_examples:
        messages.extend([
            {"role": "user", "content": f"Analysiere:\n{ex['user']}"},
            {"role": "assistant", "content": f"Zusammenfassung: {ex['assistant']}"}
        ])

    messages.append({
        "role": "user",
        "content": f"Analysiere:\n{example['prompt']}\n\nZusammenfassung:"
    })

    example["text"] = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return example

In [ ]:
# turn dataset into chat format
dataset = dataset.map(lambda example: format_chat_template(example, tokenizer))


In [ ]:
# this funciton just serves as a way to check the input of the LLM
def pretty_print_series_row(dataset, row_number):
    # ensure the input is a Hugging Face Dataset
    if not isinstance(dataset, Dataset):
        raise ValueError("Input must be a Hugging Face Dataset")

    # access the row using the row_number
    row = dataset[row_number]

    # assuming the row is a dictionary-like object
    text = row.get('text', '')  # replace 'text' with the actual column name containing the text

    # format the text, to improve readability
    formatted = (text
                 .replace('\\n', '\n')
                 .replace('- ', '  - ')
                )
    print(formatted)

pretty_print_series_row(dataset, row_number = 1)

In [ ]:
# generate a clean response
def clean_response(response):
    # exclude medical information
    response = re.sub(r'\d+[.,]\d+|\d+-\d+|\d+[/]\d+', '', response)
    response = re.sub(r'\b(Krankenhaus|Medikamente|Survey)\b', '', response, flags=re.I)
    # force correct
    if re.search(r'\d+-\d+', response) and 'jährig' not in response:
        age_match = re.search(r'(\d+-\d+)', response)
        response = f"{age_match.group(1)}-jährige " + response.replace(age_match.group(1), '')
    return response

# generate the persona (turn input into a single sentence)
def generate_persona(pipe, text, max_new_tokens=60):
    base_info = text.split('Person:')[-1].split('<|assistant|>')[0].strip()
    prompt = f"""Erstelle einen präzisen Satz nach diesem Muster:
        Angaben: Geschlecht: Weiblich, Alter: 30-39, Bildung: Hoch, Bundesland: Wien, Impfen: Trifft eher zu
        Satz: Die 30-39-jährige hochgebildete Frau aus Wien tendiert zu einer baldigen Impfung.

        Verwende diese Angaben:
        {base_info}
        Satz:"""

    outputs = pipe(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=True, # sample the next word from probability distribution
        temperature=0.2, # introduce "playfullness", lower = sharper, model does more strictly what it was advised to do
        top_k=10, # consider only the top 40 words as the next word
        top_p=0.60, # choose one of the words, which compromise 90% of the probability
        repetition_penalty=1.2,# this parameter aims to prevent the model from repeating itself. 1 no penalty, value > 1 increase the penalty, making the model less likely to repeat
        no_repeat_ngram_size=2, # do not only use 2 words in the output
        num_beams=5  # create two text-variants and choose the best one
    )

    raw_response = outputs[0]['generated_text'].split("Zusammenfassung:")[-1].strip()

    # Bereinige die Antwort
    response = clean_response(raw_response)
    print("Prompt:\n", prompt)
    print("\nAntwort:\n", response)
    return response

In [ ]:
# load model and tokenizer
model_name = "dbmdz/german-gpt2"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float32
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
# generate pipeline
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={
        "pad_token_id": tokenizer.eos_token_id
    }
)


In [ ]:
# generate personas:
new_texts = []
for idx in range(len(dataset)):
    response = generate_persona(pipe, dataset[idx]["text"])
    updated_text = dataset[idx]["text"].replace("<|assistant|>", f"<|assistant|>\n{response}</s>")
    new_texts.append(updated_text)

# create new dataset with generated personas
# dataset = dataset.add_column("text_with_response", new_texts)

# Delete?

In [ ]:
from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer

# Remove GPU-specific configs
training_args = TrainingArguments(
    output_dir="./cpu-finetuned",
    num_train_epochs=1,  # Start with 1 epoch
    per_device_train_batch_size=1,  # Batch size 1 for CPU
    learning_rate=1e-5,  # Lower learning rate
    logging_steps=5,
    save_strategy="no",  # Disable saving to save RAM
    report_to="none",
    use_cpu=True  # Force CPU usage
)

# Simplified LoRA config
lora_config = LoraConfig(
    r=8,  # Smaller rank
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)


In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

pipe = pipeline(
    "text-generation",
    model=model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

prompt = "Write a Python function that can clean the HTML tags from the file:"

outputs = pipe(
    prompt,
    max_new_tokens=300,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
)
print(outputs[0]["generated_text"])